In [19]:
!pip install pandas numpy matplotlib seaborn scikit-learn openpyxl





[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Libraries ready")


✅ Libraries ready
